In [0]:
import time
import torch
torch.__version__
torch.cuda.is_available()

Out[1]: True

In [0]:
print("Executor memory: ", spark.conf.get("spark.executor.memory"))

Executor memory:  199584m


In [0]:
import pandas as pd
from torchvision import transforms
from torchvision.models import resnet50, ResNet50_Weights

In [0]:
# Enable Arrow support.
spark.conf.set("spark.sql.execution.arrow.enabled", "true")

In [0]:
# Create and broadcast model state to all executors
model_state = resnet50(weights=ResNet50_Weights.DEFAULT).state_dict()
bc_model_state = sc.broadcast(model_state)

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

In [0]:
input_data = spark.read.format("parquet").load("s3://air-example-data-2/10G-image-data-synthetic-raw-parquet")

In [0]:
# Preprocessing
from pyspark.sql.functions import col, pandas_udf
from pyspark.sql.types import ArrayType, FloatType

import numpy as np

import torch
import time

# Read documentation here: https://spark.apache.org/docs/3.0.1/sql-pyspark-pandas-with-arrow.html

@pandas_udf(ArrayType(FloatType()))
def preprocess(image: pd.Series) -> pd.Series:
    preprocess = transforms.Compose(
        [
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ]
    )
    print(f"number of images: {len(image)}")
    # Spark has no tensor support, so it flattens the image tensor to a single array during read.
    # Each image is represented as a flattened numpy array.
    # We have to reshape back to the original number of dimensions.
    # Need to convert to float dtype otherwise torchvision transforms will complain. The data is read as short (int16) by default
    batch_dim = len(image)
    numpy_batch = np.stack(image.values)
    reshaped_images = numpy_batch.reshape(batch_dim, 256, 256, 3).astype(np.float)
    
    torch_tensor = torch.Tensor(reshaped_images.transpose(0, 3, 1, 2))
    preprocessed_images = preprocess(torch_tensor).numpy()
    # Arrow only works with single dimension numpy arrays, so need to flatten the array before outputting it
    preprocessed_images = [image.flatten() for image in preprocessed_images]
    return pd.Series(preprocessed_images)

In [0]:
preprocessed_data = input_data.select(preprocess(col("image")))

In [0]:
# 1000 is the largest batch size that can fit on GPU. Limit batch size to 1000 to avoid CUDA OOM.
spark.conf.set("spark.sql.execution.arrow.maxRecordsPerBatch", "1000")

In [0]:
@pandas_udf(ArrayType(FloatType()))
def predict(preprocessed_images: pd.Series) -> pd.Series:
    with torch.inference_mode():
        model = resnet50()
        model.load_state_dict(bc_model_state.value)
        model = model.to(torch.device("cuda")) # Move model to GPU
        model.eval()
        
        batch = preprocessed_images
        batch_dim = len(batch)
        numpy_batch = np.stack(batch.values)
        # Spark has no tensor support, so it flattens the image tensor to a single array during read.
        # Each image is represented as a flattened numpy array.
        # We have to reshape back to the original number of dimensions.
        reshaped_images = numpy_batch.reshape(batch_dim, 3, 224, 224)
        gpu_batch = torch.Tensor(reshaped_images).to(torch.device("cuda"))
        predictions = list(model(gpu_batch).cpu().numpy())
        assert len(predictions) == batch_dim
        
        return pd.Series(predictions)

In [0]:
# Repartition to 1 to limit parallelism for GPU prediction.
# Single node clusters do not have GPU scheduling support.
# Otherwise 32 tasks will run in parallel causing Cuda OOM.
one_partition_data = preprocessed_data.repartition(1)
predictions = one_partition_data.select(predict(col("preprocess(image)")))

In [0]:
start_time = time.time()
predictions.write.mode("overwrite").format("noop").save()
end_time = time.time()
print(f"Prediction took: {end_time-start_time} seconds")

assert preprocessed_data.count() == 16232

Prediction took: 137.95954656600952 seconds
